In [ ]:
import pandas as pd
import useful_functions as uf
path_fill = 'data/2022/filled_trian.csv'
df_train = pd.read_csv('data/2022/train.csv', index_col=0).head(500)
df_train_filled = pd.read_csv(path_fill, index_col=0).head(500)

distribution review